# IIEL Notebook
This notebook contains the IIEL toolkit. Run cells sequentially.

In [1]:
# IIEL - Interactive ETL, Model & Causal Toolkit (Jupytext-style single-cell script)
# This notebook contains: synthetic data generator, ETL, distance cache, kernel-based treatment,
# FE panel and event-study, power simulation, visualization helpers, and a Tkinter GUI.
# Run cells sequentially. For GUI use, run as script: python notebook_IIEL.py
# The notebook is designed to be educational and runnable in a local Jupyter environment.

In [2]:
# Cell 1 - Imports and utilities
import os
import sys
import math
import threading
import tempfile
import webbrowser
from datetime import datetime

import numpy as np
import pandas as pd

# Optional imports with graceful fallback
try:
    import geopandas as gpd
    from shapely.geometry import Point
except Exception:
    gpd = None

try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
except Exception:
    sm = None

try:
    from linearmodels.panel import PanelOLS
except Exception:
    PanelOLS = None

try:
    import matplotlib.pyplot as plt
except Exception:
    plt = None

# GUI
try:
    import tkinter as tk
    from tkinter import filedialog, messagebox, ttk
except Exception:
    tk = None

# For safe expression evaluation
try:
    import numexpr as ne
except Exception:
    ne = None

# Utilities
def haversine(lon1, lat1, lon2, lat2):
    # returns km
    R = 6371.0
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2*math.asin(math.sqrt(a))
    return R * c

In [3]:
# Cell 2 - Synthetic data generator
def generate_synthetic_data(n_munis=100, n_plants=3, years=range(2005,2021)):
    # municipios
    np.random.seed(0)
    munis = []
    for i in range(n_munis):
        lon = -3 + np.random.rand()*6  # Iberian lon rough
        lat = 40 + np.random.rand()*6
        area = np.random.uniform(20,500)
        pop = np.random.randint(500, 50000)
        wage = np.random.uniform(10000,40000)
        munis.append({'m_id': i, 'name': f'muni_{i}', 'lon': lon, 'lat': lat, 'area_km2': area,
                      'population': pop, 'wage': wage})
    munis_df = pd.DataFrame(munis)

    # plantas
    plants = []
    for p in range(n_plants):
        lon = -3 + np.random.rand()*6
        lat = 40 + np.random.rand()*6
        cap = np.random.uniform(500,1500)
        commissioning = np.random.randint(1990,2010)
        const_years = np.random.randint(2,6)
        capex = np.random.uniform(1e8,1e9)
        plants.append({'p_id': p, 'lon': lon, 'lat': lat, 'cap_MW': cap,
                       'commissioning_year': commissioning, 'construction_years': const_years,
                       'capex_total': capex})
    plants_df = pd.DataFrame(plants)

    # producción anual por planta
    records = []
    for p in plants_df.itertuples():
        for y in years:
            prod = p.cap_MW * 8760 * 0.8 if y >= p.commissioning_year else 0
            prod = prod * (1 + np.random.normal(0,0.05))
            records.append({'p_id': p.p_id, 'year': y, 'prod_MWh': max(0, prod)})
    prod_df = pd.DataFrame(records)

    # demographics yearly for municipalities
    dem_records = []
    for m in munis_df.itertuples():
        for y in years:
            pop = int(m.population * (1 + np.random.normal(0,0.01)))
            wage = m.wage * (1 + np.random.normal(0,0.02))
            dem_records.append({'m_id': m.m_id, 'year': y, 'population': pop, 'wage': wage, 'area_km2': m.area_km2})
    dem_df = pd.DataFrame(dem_records)

    return munis_df, plants_df, prod_df, dem_df

In [4]:
# Cell 2 - DataStore and ETL
class DataStore:
    def __init__(self):
        self.munis = None
        self.plants = None
        self.production = None
        self.demographics = None
        self.distance_cache = None
        self.results = None

    def load_csvs(self, munis_path=None, plants_path=None, prod_path=None, demo_path=None):
        if munis_path:
            self.munis = pd.read_csv(munis_path)
        if plants_path:
            self.plants = pd.read_csv(plants_path)
        if prod_path:
            self.production = pd.read_csv(prod_path)
        if demo_path:
            self.demographics = pd.read_csv(demo_path)

    def set_synthetic(self, munis_df, plants_df, prod_df, dem_df):
        self.munis = munis_df.copy()
        self.plants = plants_df.copy()
        self.production = prod_df.copy()
        self.demographics = dem_df.copy()

    def compute_distance_cache(self):
        if self.munis is None or self.plants is None:
            raise ValueError('Munis or plants missing')
        cache = []
        for m in self.munis.itertuples():
            for p in self.plants.itertuples():
                d = haversine(m.lon, m.lat, p.lon, p.lat)
                cache.append({'m_id': m.m_id, 'p_id': p.p_id, 'dist_km': d})
        self.distance_cache = pd.DataFrame(cache)
        return self.distance_cache

store = DataStore()

In [5]:
# Cell 3 - Kernels and treatment construction
def kernel_inv_power(dist, alpha=1.5, d0=0.1):
    return 1.0 / np.power(dist + d0, alpha)

def kernel_gaussian(dist, beta=0.01):
    return np.exp(-beta * np.power(dist, 2))

def build_treatment_intensity(store: DataStore, kernel='inv', param1=1.5, param2=0.1):
    if store.distance_cache is None:
        store.compute_distance_cache()
    dc = store.distance_cache.merge(store.plants[['p_id','cap_MW']], on='p_id', how='left')
    # join production to get prod by plant-year
    prod = store.production.copy()
    # for simplicity compute average prod per plant
    avg_prod = prod.groupby('p_id')['prod_MWh'].mean().reset_index().rename(columns={'prod_MWh':'avg_prod'})
    dc = dc.merge(avg_prod, on='p_id', how='left')
    if kernel == 'inv':
        dc['w'] = kernel_inv_power(dc['dist_km'], alpha=param1, d0=param2)
    else:
        dc['w'] = kernel_gaussian(dc['dist_km'], beta=param1)
    dc['contrib'] = dc['w'] * dc['avg_prod']
    # aggregate per muni
    T = dc.groupby('m_id')['contrib'].sum().reset_index().rename(columns={'contrib':'T_intensity'})
    # merge into demographics per year to form panel: assume same intensity across years (can be time-varying with prod)
    dem = store.demographics.copy()
    dem = dem.merge(T, on='m_id', how='left')
    dem['T_intensity'] = dem['T_intensity'].fillna(0)
    store.demographics = dem
    return dem

In [6]:
# Cell 4 - Econometric models and tests
def run_fe_panel(store: DataStore, y_var='population', cluster='m_id', add_controls=None):
    # simple FE via entity and time dummies if PanelOLS not available
    dem = store.demographics.copy()
    if add_controls is None:
        add_controls = []
    dem = dem.dropna(subset=[y_var,'T_intensity'])
    dem['logY'] = np.log1p(dem[y_var])
    dem['logT'] = np.log1p(dem['T_intensity'])
    if PanelOLS is not None:
        # prepare multiindex
        dem2 = dem.set_index(['m_id','year'])
        exog = sm.add_constant(dem2[['logT'] + add_controls])
        mod = PanelOLS(dem2['logY'], exog, entity_effects=True, time_effects=True)
        res = mod.fit(cov_type='clustered', cluster_entity=True)
        store.results = res
        return res
    else:
        # create dummies
        dem = pd.get_dummies(dem, columns=['m_id','year'], drop_first=True)
        ind = ['logT'] + add_controls + [c for c in dem.columns if c.startswith('m_id_') or c.startswith('year_')]
        X = dem[ind]
        X = sm.add_constant(X)
        y = dem['logY']
        ols = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': dem[cluster]})
        store.results = ols
        return ols

def run_event_study(store: DataStore, event_year_col='event_year', window=5):
    # Simplified: build indicators for t - event
    dem = store.demographics.copy()
    if event_year_col not in dem.columns:
        raise ValueError('event year column not present in demographics')
    dem['rel_year'] = dem['year'] - dem[event_year_col]
    # build dummies for rel_year in [-window, +window]
    for s in range(-window, window+1):
        dem[f'leadlag_{s}'] = (dem['rel_year']==s).astype(int)
    # regress logY on these dummies (omit -1 as base)
    inds = [f'leadlag_{s}' for s in range(-window, window+1) if s!=-1]
    dem['logY'] = np.log1p(dem['population'])
    X = sm.add_constant(dem[inds])
    y = dem['logY']
    ols = sm.OLS(y, X).fit()
    store.event_study = ols
    return ols

def moran_test_placeholder(store: DataStore):
    # Placeholder: recommend pysal implementation; here compute naive spatial autocorr on residuals
    return {'moran_I': None, 'p_value': None}

In [7]:
# Cell 5 - Power simulation (MDE)
def power_simulation(store: DataStore, effect_size=0.02, n_sims=200, alpha=0.05):
    # crude sim: bootstrap residuals and add effect_size to treated units
    dem = store.demographics.copy()
    # treat if T_intensity > median
    dem['treated'] = (dem['T_intensity']>dem['T_intensity'].median()).astype(int)
    base_var = np.var(np.log1p(dem['population']))
    # treat effect in log terms
    beta = effect_size
    rejects = 0
    for _ in range(n_sims):
        samp = dem.sample(frac=1, replace=True)
        samp['y'] = np.log1p(samp['population']) + beta * samp['treated'] + np.random.normal(0, np.sqrt(base_var), size=len(samp))
        X = sm.add_constant(samp['treated'])
        res = sm.OLS(samp['y'], X).fit()
        p = res.pvalues.get('treated', 1.0)
        if p < alpha:
            rejects += 1
    power = rejects / n_sims
    return power

In [ ]:
# Cell 6 - Visualization helpers
def plot_event_study(res, window=5):
    if plt is None:
        print('matplotlib not available')
        return
    coefs = res.params.filter(like='leadlag_')
    ses = res.bse.filter(like='leadlag_')
    xs = [int(c.split('_')[-1]) for c in coefs.index]
    xs_sorted = sorted(xs)
    ys = [coefs[f'leadlag_{x}'] for x in xs_sorted]
    errs = [ses[f'leadlag_{x}'] for x in xs_sorted]
    plt.errorbar(xs_sorted, ys, yerr=errs, fmt='o-')
    plt.axvline(-1, color='k', linestyle='--')
    plt.xlabel('Years relative to event')
    plt.ylabel('Coefficient (log Y)')
    plt.title('Event study')
    plt.show()

# map via simple html with points
def save_simple_map(store: DataStore, out_html='map.html'):
    try:
        import folium
    except Exception:
        print('folium not available')
        return None
    if store.munis is None or store.plants is None:
        return None
    # center map
    avg_lat = pd.concat([store.munis['lat'], store.plants['lat']]).mean()
    avg_lon = pd.concat([store.munis['lon'], store.plants['lon']]).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=7)
    # add munis colored by T_intensity (average over years)
    if 'T_intensity' in store.demographics.columns:
        tavg = store.demographics.groupby('m_id')['T_intensity'].mean().reset_index()
        munis = store.munis.merge(tavg, on='m_id', how='left')
    else:
        munis = store.munis.copy()
        munis['T_intensity'] = 0
    for r in munis.itertuples():
        folium.CircleMarker([r.lat, r.lon], radius=3+float(r.T_intensity>0), popup=f"{r.name}: {r.T_intensity:.2f}").add_to(m)
    for p in store.plants.itertuples():
        folium.Marker([p.lat, p.lon], popup=f"Plant {p.p_id}", icon=folium.Icon(color='red')).add_to(m)
    m.save(out_html)
    return out_html

In [ ]:
# Cell 7 - Tkinter GUI
if True:
    try:
        import tkinter as tk
        from tkinter import filedialog, messagebox, ttk
    except Exception:
        tk = None

if tk is None:
    print('Tkinter no disponible en este entorno. Ejecuta en tu máquina local.')
else:
    class IIELApp:
        def __init__(self, root):
            self.root = root
            root.title('IIEL - Interactive ETL & Causal Toolkit')
            # frames
            frm_top = ttk.Frame(root, padding=8)
            frm_top.grid(row=0, column=0, sticky='ew')
            frm_mid = ttk.Frame(root, padding=8)
            frm_mid.grid(row=1, column=0, sticky='nsew')
            frm_bot = ttk.Frame(root, padding=8)
            frm_bot.grid(row=2, column=0, sticky='ew')

            # Top: load / synthetic
            ttk.Button(frm_top, text='Cargar CSVs...', command=self.load_csvs).grid(row=0,column=0, padx=4)
            ttk.Button(frm_top, text='Generar datos sintéticos', command=self.gen_synth).grid(row=0,column=1, padx=4)
            ttk.Button(frm_top, text='Calcular distancias', command=self.compute_dist).grid(row=0,column=2, padx=4)

            # Mid: kernel params
            ttk.Label(frm_mid, text='Kernel').grid(row=0,column=0)
            self.kernel_var = tk.StringVar(value='inv')
            ttk.Combobox(frm_mid, textvariable=self.kernel_var, values=['inv','gauss']).grid(row=0,column=1)
            ttk.Label(frm_mid, text='param1 (alpha/beta)').grid(row=1,column=0)
            self.param1 = tk.DoubleVar(value=1.5)
            ttk.Entry(frm_mid, textvariable=self.param1).grid(row=1,column=1)
            ttk.Label(frm_mid, text='param2 (d0)').grid(row=2,column=0)
            self.param2 = tk.DoubleVar(value=0.1)
            ttk.Entry(frm_mid, textvariable=self.param2).grid(row=2,column=1)
            ttk.Button(frm_mid, text='Construir treatment', command=self.build_treatment).grid(row=3,column=0, columnspan=2, pady=6)

            # Bottom: modeling and viz
            ttk.Button(frm_bot, text='Run FE panel (population)', command=self.run_panel).grid(row=0,column=0, padx=4)
            ttk.Button(frm_bot, text='Event study (requires event_year col)', command=self.run_event).grid(row=0,column=1, padx=4)
            ttk.Button(frm_bot, text='Salvar mapa HTML', command=self.save_map).grid(row=0,column=2, padx=4)
            ttk.Button(frm_bot, text='Exportar resultados CSV', command=self.export_results).grid(row=0,column=3, padx=4)

            # status
            self.status = tk.StringVar(value='Idle')
            ttk.Label(root, textvariable=self.status).grid(row=3,column=0, sticky='w')

        def set_status(self, txt):
            self.status.set(txt)
            self.root.update_idletasks()

        def load_csvs(self):
            # ask for four files in sequence
            try:
                munis_path = filedialog.askopenfilename(title='CSV municipios', filetypes=[('CSV','*.csv')])
                plants_path = filedialog.askopenfilename(title='CSV plantas', filetypes=[('CSV','*.csv')])
                prod_path = filedialog.askopenfilename(title='CSV produccion', filetypes=[('CSV','*.csv')])
                demo_path = filedialog.askopenfilename(title='CSV demografics', filetypes=[('CSV','*.csv')])
                store.load_csvs(munis_path, plants_path, prod_path, demo_path)
                self.set_status('CSV cargados')
            except Exception as e:
                messagebox.showerror('Error carga', str(e))

        def gen_synth(self):
            munis_df, plants_df, prod_df, dem_df = generate_synthetic_data()
            store.set_synthetic(munis_df, plants_df, prod_df, dem_df)
            self.set_status('Datos sintéticos generados')

        def compute_dist(self):
            try:
                store.compute_distance_cache()
                self.set_status('Distances computed')
            except Exception as e:
                messagebox.showerror('Error distances', str(e))

        def build_treatment(self):
            try:
                k = self.kernel_var.get()
                p1 = float(self.param1.get())
                p2 = float(self.param2.get())
                dem = build_treatment_intensity(store, kernel=k, param1=p1, param2=p2)
                self.set_status('Treatment built')
                messagebox.showinfo('Success', 'Treatment constructed and merged into demographics')
            except Exception as e:
                messagebox.showerror('Error build', str(e))

        def run_panel(self):
            try:
                res = run_fe_panel(store, y_var='population')
                self.set_status('FE panel run')
                # show summary in window
                txt = str(res.summary()) if hasattr(res, 'summary') else str(res)
                win = tk.Toplevel(self.root); win.title('FE results')
                txtw = tk.Text(win, wrap='none', width=120, height=30)
                txtw.insert('1.0', txt)
                txtw.pack(fill='both', expand=True)
            except Exception as e:
                messagebox.showerror('Error panel', str(e))

        def run_event(self):
            try:
                res = run_event_study(store)
                self.set_status('Event study run')
                plot_event_study(res)
            except Exception as e:
                messagebox.showerror('Error event', str(e))

        def save_map(self):
            try:
                out = save_simple_map(store, out_html='iiel_map.html')
                if out:
                    webbrowser.open('file://' + os.path.abspath(out))
                    self.set_status('Map saved and opened')
                else:
                    self.set_status('Map not created')
            except Exception as e:
                messagebox.showerror('Error map', str(e))

        def export_results(self):
            try:
                if store.demographics is not None:
                    path = filedialog.asksaveasfilename(defaultextension='.csv')
                    store.demographics.to_csv(path, index=False)
                    self.set_status('Exported demographics')
                else:
                    messagebox.showinfo('No data', 'No demographics to export')
            except Exception as e:
                messagebox.showerror('Error export', str(e))

    def launch_app():
        root = tk.Tk()
        app = IIELApp(root)
        root.mainloop()

# If run as script, start GUI
if __name__ == '__main__':
    # safe start
    try:
        import tkinter as tk
    except Exception:
        tk = None
    if tk is None:
        print('Tkinter not available. Run in local Python with tkinter support.')
    else:
        launch_app()